## Download the video

We're going to use [yt-dlp](https://github.com/yt-dlp/yt-dlp) to download a video. It's not very fun to use so I recommend taking any troubles/questions to ChatGPT. It's a pretty popular piece of software so ChatGPT usually has good answers.

First we'll install it...

In [ ]:
%pip install --quiet --upgrade "yt-dlp[default]"

...then we'll use it to download [this video](https://www.youtube.com/watch?v=uvQK7WTkKpI). If you want a different video, you just change the URL inside of the quotes. It also works for TikTok!

In [ ]:
!yt-dlp "https://www.youtube.com/watch?v=uvQK7WTkKpI"

## Transcribe the video

I'm going to use [insanely fast whisper](https://github.com/Vaibhavs10/insanely-fast-whisper) instead of the normal one so I can show you how fast it can be on Google Colab!

> But honestly: just use EasyWhisper (Windows) or MacWhisper (macOS). Unless you're writing software there's no real reason to be doing this stuff with Python!

In [ ]:
%pip install --quiet --upgrade insanely-fast-whisper

The code below will only work if you are on Colab or have a fancy Windows computer!

In [ ]:
%pip install --quiet --upgrade flash-attn

If you're running on Google Colab, with a high-quality model (recommended):

```
!insanely-fast-whisper \
    --flash True \
    --file-name "How to Fix Holes in Drywall - 4 Easy Methods [uvQK7WTkKpI].webm"
```

If you're running on macOS, with a smaller/faster/worse quality model:

```
!insanely-fast-whisper \
    --device-id mps \
    --model-name openai/whisper-small.en \
    --file-name "How to Fix Holes in Drywall - 4 Easy Methods [uvQK7WTkKpI].webm"
```

Again, just use MacWhisper or EasyWhisper for this!!! Unless you really need to program, there's no true need for this.

In [35]:
!insanely-fast-whisper \
    --device-id mps \
    --model-name openai/whisper-small.en \
    --file-name "How to Fix Holes in Drywall - 4 Easy Methods [uvQK7WTkKpI].webm"

/Users/soma/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
🤗 Transcribing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:01:35
Voila!✨ Your file has been transcribed go check it out over here 👉 output.json


In [44]:
import json
import pandas as pd

with open("output.json") as fp:
    data = json.load(fp)

df = pd.json_normalize(data['chunks'])
df['start'] = df.timestamp.str[0]
df['end'] = df.timestamp.str[1]
df = df.drop(columns='timestamp')
df.to_csv("segments.csv", index=False)

text = '\n'.join([chunk['text'] for chunk in data['chunks']])
with open("transcript.txt", 'w') as fp:
    fp.write(text)

In [45]:
print(text[:1000])

 Hi there, I'm Nils with Learn2DIY and sometimes bad stuff happens to your drywall.
 So today we're going to cover all of these types of damage and we're going to
 show you how to fix each one of them.
 Now just about every hole that you'll find in drywall falls into one of four
 types. Basically you've got something that's really small,
 like an inch, inch and a half or smaller,
 maybe that's two and a half, three centimeters.
 And that's something that we can patch a certain way up from there.
 You got anything between that size and maybe fist size,
 four or five inches or so.
 And then from there you've got something that's big enough that it's in between
 your studs or maybe it's even over a stud, but it needs some support.
 So we're talking stuff that's about this big or smaller, maybe a seven,
 eight inches, nine inches in that range.
 And then lastly we've got some major damage like what we showed over here.
 And this is just going to need to have a whole section replaced.
 So t

## Summarize with OpenAI

We *could* drop this into ChatGPT manually or use Google Sheets with Claude for Sheets, but instead we're going to keep going with coding. Unlike Whisper which ran on our own computer, this does require us to send the content to OpenAI to get a summary.

You'll need to add an API key below to get access to OpenAI through the software.

In [ ]:
%pip install --quiet --upgrade openai

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='sk-proj-XXXXXXXXXX')

text = open("transcript.txt").read()

result = client.chat.completions.create(
    messages=[{ "role": "user", "content": f"Summarize the transcript below:\n\n{text}",}],
    model="gpt-4o",
    max_tokens=1024
)

In [46]:
summary = result.choices[0].message.content
print(summary)

Nils from Learn2DIY outlines how to fix different types of drywall damage, categorized into four main types: 

1. **Small Holes (up to 1.5 inches/3.8 cm):**
   - Use mud to fill in these tiny holes without needing tape or a patch kit. Generic mud bought from a store is sufficient.
   - Apply mud with a putty knife, ensuring it packs firmly.

2. **Medium Holes (up to fist-sized, around 4-5 inches/10-12.7 cm):**
   - Three methodologies: 
     - **Mesh Adhesive Patch:** Smooth out over the hole, apply mud generously to cover.
     - **Presto Patch:** Cut a template, fit in the patch, and apply mud around and within the patch.
     - **California Patch:** Make a custom patch from drywall, apply mud, and fit it in just like the Presto Patch.

3. **Larger Holes (up to 10 inches/25.4 cm):**
   - Require backer boards for extra support.
   - Clean the edges, use some wood as backer boards, screw them in for support, and then fit the patch.

4. **Major Damage (large sections):**
   - Cut from 